# Setup

In [1]:
### Packages import
import os
import gc
import time
import numpy as np
from time import strftime
import sys

import torch
from torchvision import models
from src.cuda_checker import cuda_torch_check, memory_checker

### My modules import
from src.data_loader import argObj, data_loaders_stimuli_fmri
from src import image_preprocessing
from src.feature_extraction import model_loader, fit_pca, pca_batch_calculator, extract_and_pca_features, extract_features_no_pca
from src.encoding import linear_regression, compute_perason_numpy
from src.evaluation_metrics import median_squared_noisenorm_correlation
from src.visualize import histogram, box_plot, noise_norm_corr_ROI, final_subj_corr_dataframe_boxplot_istograms

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from scipy.stats import pearsonr
from src.visualize import histogram, box_plot

### Cuda setup and check
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

# Select the device to run the model on
device = 'cuda' #@param ['cpu', 'cuda'] {allow-input: true}
# Check if cuda is available
device = torch.device(device)
cuda_torch_check()

Check if GPU is available and if torch is using it ..


Torch Cuda is available?
True
Torch Cuda device count is :
1
Torch Cuda current device is :
0
Torch Cuda device is :
NVIDIA GeForce RTX 3070 Laptop GPU
Pytorch version：
1.13.0
CUDA Version: 
11.6
cuDNN version is :
8302




In [2]:
train_percentage = 90 # X% of the training data will be used for training, (100-X)% for validation
transform = image_preprocessing.imagenet_V2_transform

batch_size = 32
pca_component = 1024
min_pca_batch_size = pca_component + 300 # pca_component * 2

compute_pca = True
architecture = "EndtoEnd" #@param ["EndtoEnd", "TwoStep"] {allow-input: true}
feature_model_type = "VGG19" #@param ["alexnet", "ZFNet", "resnet50", "vgg16","vgg19_bn" , "efficientnetb2", "efficientnetb2lib"]
model_layer = "features.43"
regression_type = "MLP" #@param ["linear", "ridge"]

save = False 

alpha_l = 1e5
alpha_r = 1e5
grid_search = False

### Path definition
if isinstance(model_layer, list):
    model_layer_full = '+'.join(model_layer)
else:
    model_layer_full = model_layer

datetime_id = strftime("(%Y-%m-%d_%H-%M)")
submission_name = f'{strftime("(%Y-%m-%d_%H-%M)")}-{architecture}_{feature_model_type}_{model_layer}-pca_{pca_component}-{regression_type}-alpha_{"{:.1e}".format(alpha_l)}'

data_home_dir = '../Datasets/Biomedical'
data_dir = '../Datasets/Biomedical/algonauts_2023_challenge_data'
# Used to save the prediction of saved model
parent_submission_dir = f'./files/submissions/{submission_name}'
if not os.path.isdir(parent_submission_dir) and save:
            os.makedirs(parent_submission_dir)
ncsnr_dir = '../Datasets/Biomedical/algonauts_ncsnr'
images_trials_dir = '../Datasets/Biomedical/algonauts_train_images_trials'

In [3]:
subj = 1

In [4]:
submission_name

'(2023-06-19_15-03)-EndtoEnd_VGG19_features.43-pca_1024-MLP-alpha_1.0e+05'

In [5]:
print('############################ Subject: ' + str(subj) + ' ############################ \n')
# Definining paths to data and submission directories ##
args = argObj(subj, data_home_dir, data_dir, parent_submission_dir, ncsnr_dir, images_trials_dir, save) 
# Obtain the indices of the training, validation and test data
idxs_train, idxs_val, idxs_test, train_imgs_paths, test_imgs_paths = args.images_idx_splitter(train_percentage)

# Defining the images data loaderds
data_loaders = data_loaders_stimuli_fmri(idxs_train, 
                                            idxs_val, 
                                            idxs_test, 
                                            train_imgs_paths, 
                                            test_imgs_paths,
                                            lh_fmri_path = args.lh_fmri,
                                            rh_fmri_path = args.rh_fmri)

train_dataloader_lh, val_dataloader_lh, test_dataloader_lh, train_dataloader_rh, val_dataloader_rh, test_dataloader_rh = data_loaders.images_fmri_dataloader(batch_size, transform)

lh_fmri_train, lh_fmri_val, rh_fmri_train, rh_fmri_val = data_loaders.fmri_splitter()

############################ Subject: 1 ############################ 

## Stimulus Images Loading: Info
Total train images: 9841
Training stimulus images: 8857
Validation stimulus images: 984
Test stimulus images: 159




# Custom Net

##  Input Exploration

In [6]:
first_batch = next(iter(train_dataloader_lh))


In [7]:
first_batch[0].shape

torch.Size([32, 3, 224, 224])

In [8]:
first_batch[1].shape[1]

19004

## Architettura

In [9]:
# Definizione della rete custom
class CustomNet(nn.Module):
    def __init__(self, num_outputs):
        super(CustomNet, self).__init__()
        
        # Caricamento del modello VGG-19 pre-addestrato
        # vgg = models.vgg19_bn(pretrained=True)
        
        # Utilizziamo solo i layer di feature extraction senza gli ultimi layer completamente connessi
        self.features = nn.Sequential(*list(models.alexnet(pretrained=True).features.children()))
        for param in self.features.parameters():
            param.requires_grad = False
        
        # Aggiunta del layer di average pooling
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(6, 6))
        
        # Aggiunta del layer completamente connesso per la regressione
        self.fc = nn.Linear(9216, num_outputs)  # L'input size 512 è determinato dal numero di feature maps generate dall'ultimo layer di VGG-19
        
    def forward(self, x):
        x = self.features(x)
        #x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [10]:
# Creazione dell'istanza del modello
model = CustomNet(first_batch[1].shape[1])
model.to(device)
model.train()
for param in model.features.parameters():
    param.requires_grad = False
# Stampa della struttura del modello
print(model)

c:\Users\giorg\miniconda3\envs\algonauts_test\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\giorg\miniconda3\envs\algonauts_test\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=9216, out_features=19004, bias=True)
)


In [11]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, requires_grad: {param.requires_grad}')

Layer: features.0.weight, requires_grad: False
Layer: features.0.bias, requires_grad: False
Layer: features.3.weight, requires_grad: False
Layer: features.3.bias, requires_grad: False
Layer: features.6.weight, requires_grad: False
Layer: features.6.bias, requires_grad: False
Layer: features.8.weight, requires_grad: False
Layer: features.8.bias, requires_grad: False
Layer: features.10.weight, requires_grad: False
Layer: features.10.bias, requires_grad: False
Layer: fc.weight, requires_grad: True
Layer: fc.bias, requires_grad: True


In [12]:
print(model.features[-3].weight.grad)
print(model.fc.weight.grad)

None
None


## Training

In [13]:
# Metrica da printare
def mean_pearson_corr(y_true, y_pred):
    pearson_corr = 0.0
    for i in range(y_true.shape[1]):
        corr, _ = pearsonr(y_true[:, i], y_pred[:, i])
        if corr < 0:
            corr = 0.0
        if corr > 1:
            corr = 1.0
        pearson_corr += corr
    return pearson_corr / y_true.shape[1]

Parametri di training

In [14]:
learning_rate = 0.001
num_epochs = 10

Clean

In [53]:
# model.to('cpu') # sposto sulla ram
# del model
# torch.cuda.empty_cache() # elimino la chache vram
# gc.collect()

In [15]:
# Definizione della loss function
criterion = nn.MSELoss()

# Selezionare solo i parametri del layer regressor per l'ottimizzazione
# parameters = model.fc.parameters()

# Definizione dell'ottimizzatore
# optimizer = optim.Adam(parameters, lr=learning_rate)

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

In [16]:
from tqdm import tqdm

for epoch in range(num_epochs):
    #model.train()
    total_loss = 0.0
    total_rmse = 0.0
    total_mae = 0.0
    total_pearson_corr = 0.0
    total_batches = 0
    
    for images, labels in tqdm(train_dataloader_lh):
        #print(labels.shape)
        
        # Reset dei gradienti
        optimizer.zero_grad()
        
        # Passaggio degli input attraverso la rete
        outputs = model(images.to(device))
        
        # Calcolo della loss
        loss = criterion(outputs, labels.to(device))
        
        # Backpropagation
        loss.backward()
        
        # Clean memory
        images = images.detach().cpu()
        labels = labels.detach().cpu()
        outputs = outputs.detach().cpu()
        torch.cuda.empty_cache()
        
        # Ottimizzazione dei parametri
        optimizer.step()
        
        # Calcolo delle metriche di accuratezza
        batch_size = images.size(0)
        total_loss += loss.item() * batch_size
        total_rmse += mean_squared_error(labels.detach().cpu().numpy(), outputs.detach().cpu().numpy(), squared=False) * batch_size
        total_mae += mean_absolute_error(labels.detach().cpu().numpy(), outputs.detach().cpu().numpy()) * batch_size
        total_pearson_corr += mean_pearson_corr(labels.detach().cpu().numpy(), outputs.detach().cpu().numpy()) * batch_size
        total_batches += batch_size
    
    # Calcolo delle metriche di accuratezza medie
    average_loss = total_loss / total_batches
    average_rmse = total_rmse / total_batches
    average_mae = total_mae / total_batches
    average_pearson_corr = total_pearson_corr / total_batches
    
    # Stampa delle metriche di accuratezza
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}, RMSE: {average_rmse:.4f}, MAE: {average_mae:.4f}, Pearson Corr: {average_pearson_corr:.4f}")

100%|██████████| 277/277 [04:36<00:00,  1.00it/s]


Epoch [1/10], Loss: 1.9794, RMSE: 1.3346, MAE: 1.0721, Pearson Corr: 0.1655


100%|██████████| 277/277 [04:33<00:00,  1.01it/s]


Epoch [2/10], Loss: 2.0223, RMSE: 1.3711, MAE: 1.1010, Pearson Corr: 0.2275


100%|██████████| 277/277 [04:34<00:00,  1.01it/s]


...

Epoch [8/10], Loss: 5.5918, RMSE: 2.1434, MAE: 1.7991, Pearson Corr: 0.3013


100%|██████████| 277/277 [04:34<00:00,  1.01it/s]


Epoch [9/10], Loss: 5.7595, RMSE: 2.1586, MAE: 1.8181, Pearson Corr: 0.3113


100%|██████████| 277/277 [04:35<00:00,  1.01it/s]

Epoch [10/10], Loss: 5.4621, RMSE: 2.0880, MAE: 1.7635, Pearson Corr: 0.3306


# Evaluation

In [17]:
model.eval()

CustomNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=9216, out_features=19004, bias=True)
)

In [18]:
len(val_dataloader_lh)

31

In [19]:
lh_fmri_val_pred = None
lh_fmri_val_ground = None
# Initialize counter for stacked batches
count_stacked_batches = 0
# Define list to store downsampled features
features = []

for _, (images, labels) in tqdm(enumerate(val_dataloader_lh), total=len(val_dataloader_lh)):
    if lh_fmri_val_pred is None and lh_fmri_val_ground is None:
        with torch.no_grad():
            lh_fmri_val_pred = model(images.to(device)).detach().cpu().numpy()
        lh_fmri_val_ground = labels.numpy()
    else:
        with torch.no_grad():
            lh_fmri_val_pred = np.vstack((lh_fmri_val_pred, model(images.to(device)).detach().cpu().numpy()))
        lh_fmri_val_ground = np.vstack((lh_fmri_val_ground, labels.numpy()))

100%|██████████| 31/31 [00:09<00:00,  3.18it/s]


In [50]:
rh_fmri_val_pred = None
# Initialize counter for stacked batches
count_stacked_batches = 0
# Define list to store downsampled features
features = []

for _, (images, labels) in tqdm(enumerate(val_dataloader_rh), total=len(val_dataloader_rh)):
    if rh_fmri_val_pred is None:
        rh_fmri_val_pred = model(images.to(device)).detach().cpu().numpy()
    else:
        rh_fmri_val_pred = np.vstack((rh_fmri_val_pred, model(images.to(device)).detach().cpu().numpy()))

100%|██████████| 62/62 [00:10<00:00,  6.11it/s]


In [20]:
rh_fmri_val_pred = rh_fmri_val

In [21]:
noise_norm_corr_dict_1, noise_norm_corr_dict_2 = median_squared_noisenorm_correlation(lh_fmri_val_pred, 
                                                                                        rh_fmri_val_pred,
                                                                                        lh_fmri_val,
                                                                                        rh_fmri_val,
                                                                                        args.data_dir,
                                                                                        args.ncsnr_dir,
                                                                                        args.images_trials_dir,
                                                                                        idxs_val)

Computing the correlation between the predicted and actual fMRI data...


100%|██████████| 20544/20544 [00:01<00:00, 16406.46it/s]


In [22]:
np.median(noise_norm_corr_dict_1)

0.030948432878404552

Calcolato con la funzione

In [109]:
mean_pearson_corr(lh_fmri_val, lh_fmri_val_pred)**2

0.025723810331753576

In [40]:
noise_norm_corr_ROI_dict_1 = noise_norm_corr_ROI(args.data_dir, 
                                                                  noise_norm_corr_dict_1, 
                                                                  noise_norm_corr_dict_2,
                                                                  save)

In [48]:
lh_fmri_val_pred.shape

(984, 19004)

In [49]:
lh_fmri_val.shape

(984, 19004)

In [36]:
model(first_batch[0].to(device)).shape

torch.Size([16, 19004])

In [ ]:

model, feature_extractor = model_loader(feature_model_type, model_layer, device)

# Fit the PCA model
if compute_pca:
    # Fit the PCA model
    pca_batch_size, n_stacked_batches = pca_batch_calculator(len(idxs_train),
                                                            batch_size,
                                                            min_pca_batch_size,
                                                            pca_component)
    
    pca = fit_pca(feature_extractor,
                    train_imgs_dataloader,
                    pca_component,
                    n_stacked_batches,
                    pca_batch_size,
                    device)
    print("Comulative Explained variance ratio: ", sum(pca.explained_variance_ratio_))
    print("Number of components: ", pca.n_components_)
    
    print('## Extracting features from training, validation and test data...')
    features_train = extract_and_pca_features(feature_extractor, train_imgs_dataloader, pca, n_stacked_batches, device)
    features_val = extract_and_pca_features(feature_extractor, val_imgs_dataloader, pca, n_stacked_batches, device)
    features_test = extract_and_pca_features(feature_extractor, test_imgs_dataloader, pca, n_stacked_batches, device)
    
    # print("\n")
    # print('## Checking and Freeing  GPU memory...')
    # memory_checker()
    model.to('cpu') # sposto sulla ram
    feature_extractor.to('cpu') # sposto sulla ram
    del model, feature_extractor, pca, train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader  # elimino dalla ram
    torch.cuda.empty_cache() # elimino la chache vram
    gc.collect() # elimino la cache ram
    # memory_checker()
else:
    print('## Extracting features from training, validation and test data...')
    features_train = extract_features_no_pca(feature_extractor, train_imgs_dataloader, device)
    features_val = extract_features_no_pca(feature_extractor, val_imgs_dataloader, device)
    features_test = extract_features_no_pca(feature_extractor, test_imgs_dataloader, device)
    
    model.to('cpu') # sposto sulla ram
    feature_extractor.to('cpu') # sposto sulla ram
    del model, feature_extractor, train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader  # elimino dalla ram
    torch.cuda.empty_cache() # elimino la chache vram
    gc.collect() # elimino la cache ram

In [ ]:
lh_fmri_train, lh_fmri_val, rh_fmri_train, rh_fmri_val = data_loaders.fmri_splitter()

# Grid Search

In [4]:
## Fit the linear model ##
print('\n ## Fit Encoder and Predict...')
lh_fmri_train, lh_fmri_val, rh_fmri_train, rh_fmri_val = data_loaders.fmri_splitter()
print('LH fMRI number of vertices:', lh_fmri_train.shape)
print('RH fMRI number of vertices:', rh_fmri_train.shape)
# param_grid = {'alpha': [0.0001, 0.0002, 0.001, 0.01, 0.1, 1, 10, 100, 1e3, 5e3, 1e4, 2e4, 5e4, 1e5, 1e6]}

param_grid = {'alpha': [1, 10, 100, 1e3, 5e3, 1e4, 2e4, 5e4, 1e5, 2e5, 5e5, 1e6]}
#param_grid = {'alpha': [1e6, 2e6, 5e6, 1e7, 2e7, 5e7]}


 ## Fit Encoder and Predict...
LH fMRI number of vertices: (8857, 19004)
RH fMRI number of vertices: (8857, 20544)


In [5]:
grid_search_l = GridSearchCV(Ridge(), param_grid=param_grid, scoring=make_scorer(
    lambda x, y: np.median(compute_perason_numpy(x, y))), cv=5, n_jobs=5, verbose=1)
grid_search_l.fit(X=features_train, y=lh_fmri_train)
print("Best Param: {}".format(grid_search_l.best_params_))
print("Best Score: {}".format(grid_search_l.best_score_))
alpha_l = grid_search_l.best_params_['alpha']

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Param: {'alpha': 100000.0}
Best Score: 0.4171876543623486


In [6]:
alpha_r = alpha_l

In [9]:
print(alpha_r)

1000000.0


In [7]:
grid_search_r = GridSearchCV(Ridge(), param_grid=param_grid, scoring=make_scorer(
    lambda x, y: np.median(compute_perason_numpy(x, y))), cv=5, n_jobs=5, verbose=1)
grid_search_r.fit(X=features_train, y=rh_fmri_train)
print("Best Param: {}".format(grid_search_r.best_params_))
print("Best Score: {}".format(grid_search_r.best_score_))
alpha_r = grid_search_r.best_params_['alpha']

Fitting 5 folds for each of 10 candidates, totalling 50 fits


# Predict and evaluate 

In [7]:
lh_fmri_val_pred, lh_fmri_test_pred, rh_fmri_val_pred, rh_fmri_test_pred = linear_regression(regression_type, 
                                                                                                features_train, 
                                                                                                features_val, 
                                                                                                features_test, 
                                                                                                lh_fmri_train, 
                                                                                                rh_fmri_train, 
                                                                                                save_predictions,
                                                                                                args.subject_test_submission_dir,
                                                                                                alpha_l,
                                                                                                alpha_r,
                                                                                                grid_search= False)

noise_norm_corr_dict[f'lh_{subj}'], noise_norm_corr_dict[f'rh_{subj}'] = median_squared_noisenorm_correlation(lh_fmri_val_pred, 
                                                                                                                rh_fmri_val_pred,
                                                                                                                lh_fmri_val,
                                                                                                                rh_fmri_val,
                                                                                                                args.data_dir,
                                                                                                                args.ncsnr_dir,
                                                                                                                args.images_trials_dir,
                                                                                                                idxs_val)
print("\n Score -> Median Noise Normalized Squared Correlation Percentage (LH and RH)")
print("LH subj",subj,"| Score: ",np.median(noise_norm_corr_dict[f'lh_{subj}'])*100)
print("RH subj",subj,"| Score: ",np.median(noise_norm_corr_dict[f'rh_{subj}'])*100)

Fitting ridge regressions on the training data...
Predicting fMRI data on the validation and test data...
Computing the correlation between the predicted and actual fMRI data...


100%|██████████| 20544/20544 [00:01<00:00, 11662.10it/s]



 Score -> Median Noise Normalized Squared Correlation Percentage (LH and RH)
LH subj 1 | Score:  48.39405012388089
RH subj 1 | Score:  47.737441465482924


# Visualize

In [10]:
histogram(args.data_dir, noise_norm_corr_dict[f'lh_{subj}'], 
          noise_norm_corr_dict[f'rh_{subj}'], 
          submission_name, 
          save = args.subject_images_submission_dir)



TypeError: expected str, bytes or os.PathLike object, not bool

In [ ]:
box_plot(args.data_dir, noise_norm_corr_dict[f'lh_{subj}'], 
          noise_norm_corr_dict[f'rh_{subj}'], 
          submission_name, 
          save = args.subject_images_submission_dir)